In [29]:
import sys
import os 
import numpy as np
import pandas as pd
from scipy.stats.qmc import Sobol
from SALib.util import scale_samples
from SALib.sample import sobol
from SALib.analyze import sobol as sobol_analyze
import random

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.sampling.sampler import get_polarisation_curve_samples, build_fixed_parameters, sample_parameters, PARAMETER_RANGES
from configuration.settings import current_density_parameters, physical_parameters, computing_parameters, operating_inputs
from modules.display_modules import plot_lambda
from model.AlphaPEM import AlphaPEM


In [30]:
#del PARAMETER_RANGES['Pc_des']

In [31]:
def make_bounds_exclusive(bounds, epsilon=1e-8):
    return [(low + epsilon, high - epsilon) for (low, high) in bounds]

def prepare_problem(PARAMETER_, dependent_param="Pc_des"):
    independent_params = {
        k: v for k, v in PARAMETER_.items()
        if k != dependent_param
    }
    # Add dummy variable to later compute dependent_param
    z_key = f"{dependent_param}"
    independent_params[z_key] = (0, 1)

    bounds = [v if isinstance(v, tuple) else [min(v), max(v)] for v in independent_params.values()]
    bounds = make_bounds_exclusive(bounds)

    problem = {
        "num_vars": len(independent_params),
        "names": list(independent_params.keys()),
        "bounds": bounds,
        "groups": list(independent_params.keys())  # optional but useful for group-based SA
    }
    return problem

problem = prepare_problem(PARAMETER_=PARAMETER_RANGES,dependent_param = "Pc_des")


In [32]:
seed = 33
N=2000
np.random.seed(seed)
random.seed(seed)
samples = sobol.sample(problem, N=N, calc_second_order= False, scramble= True, seed=seed)
pd.DataFrame(samples).to_pickle("../data/raw/sobol_sampling_configurations.pkl")

d:\Users\kju10\Documents\LMU-STATISTICS & DATA SCIENCE MASTER\WS2425\Consulting\project\Sensitivity-Analysis-and-Surrogate-Modeling-of-PEM-Fuel-Cells\env\Lib\site-packages\scipy\stats\_qmc.py:958: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  sample = self._random(n, workers=workers)


In [33]:
samples = pd.read_pickle('../data/raw/sobol_sampling_configurations.pkl')
samples.columns = problem['names']
samples['Pc_des'] = samples['Pa_des']-20000
samples['e'] = np.round(samples['e'])

In [34]:
samples.head()

,Tfc,Pa_des,Sc,Phi_c_des,epsilon_gdl,tau,epsilon_mc,epsilon_c,e,Re,i0_c_ref,kappa_co,kappa_c,a_slim,b_slim,a_switch,Pc_des
0,335.015900,215267.894967,2.784156,0.607013,0.663508,2.960479,0.268524,0.237199,4.0,0.000002,449.366354,18.059504,73.490571,0.043931,0.924738,0.877939,195267.894967
1,335.930971,215267.894967,2.784156,0.607013,0.663508,2.960479,0.268524,0.237199,4.0,0.000002,449.366354,18.059504,73.490571,0.043931,0.924738,0.877939,195267.894967
2,335.015900,164712.670632,2.784156,0.607013,0.663508,2.960479,0.268524,0.237199,4.0,0.000002,449.366354,18.059504,73.490571,0.043931,0.924738,0.877939,144712.670632
3,335.015900,215267.894967,2.837283,0.607013,0.663508,2.960479,0.268524,0.237199,4.0,0.000002,449.366354,18.059504,73.490571,0.043931,0.924738,0.877939,195267.894967
4,335.015900,215267.894967,2.784156,0.340410,0.663508,2.960479,0.268524,0.237199,4.0,0.000002,449.366354,18.059504,73.490571,0.043931,0.924738,0.877939,195267.894967


In [35]:
pd.DataFrame(samples).to_pickle("../data/raw/sobol_sampling_configurations_cmpl.pkl")

## We read the saved configurations and star the simulations

In [ ]:
samples = pd.read_pickle('../data/raw/sobol_sampling_configurations_cmpl.pkl')

In [36]:
split_dejvis, split_nathaly, split_camila = np.array_split(samples, 3)

d:\Users\kju10\Documents\LMU-STATISTICS & DATA SCIENCE MASTER\WS2425\Consulting\project\Sensitivity-Analysis-and-Surrogate-Modeling-of-PEM-Fuel-Cells\env\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [41]:
print(f'shape1 {split_dejvis.shape}, shape2 {split_nathaly.shape} and shape3 {split_camila.shape}')


shape1 (12667, 17), shape2 (12667, 17) and shape3 (12666, 17)


In [ ]:
save_filepath = "../data/raw/sobol_sampling_split_camila.pkl"
results = get_polarisation_curve_samples(sampled_parameters=split_camila.to_dict(orient='records'), fixed_parameters = build_fixed_parameters(), save_path='../data/raw/sample_1_test.pkl')#, save_every=10


✅ Saved 10 samples to ../data/raw/sample_1_test.pkl
✅ Saved 20 samples to ../data/raw/sample_1_test.pkl
✅ Saved 30 samples to ../data/raw/sample_1_test.pkl
✅ Saved 40 samples to ../data/raw/sample_1_test.pkl
✅ Saved 50 samples to ../data/raw/sample_1_test.pkl
✅ Saved 60 samples to ../data/raw/sample_1_test.pkl
✅ Saved 70 samples to ../data/raw/sample_1_test.pkl
✅ Saved 80 samples to ../data/raw/sample_1_test.pkl
✅ Saved 90 samples to ../data/raw/sample_1_test.pkl
✅ Saved 100 samples to ../data/raw/sample_1_test.pkl
✅ Saved 110 samples to ../data/raw/sample_1_test.pkl
✅ Saved 120 samples to ../data/raw/sample_1_test.pkl
✅ Saved 130 samples to ../data/raw/sample_1_test.pkl
✅ Saved 140 samples to ../data/raw/sample_1_test.pkl
✅ Saved 150 samples to ../data/raw/sample_1_test.pkl
✅ Saved 160 samples to ../data/raw/sample_1_test.pkl
✅ Saved 170 samples to ../data/raw/sample_1_test.pkl
✅ Saved 180 samples to ../data/raw/sample_1_test.pkl
✅ Saved 190 samples to ../data/raw/sample_1_test.pkl
✅ 